## Imports

In [1]:
import pandas as pd
import numpy as np
import re
import os
import folium
import glob

## Data Read-in

In [2]:
# Find all CSV files in the current working directory
csv_files = glob.glob('*.csv')

# Concatenate all CSV files into a single dataframe
df = pd.concat([pd.read_csv(f) for f in csv_files], ignore_index=True)

In [3]:
df = df.loc[df['SALE TYPE'] != 'Over 500 results. Try zooming in or modifying your search options.']

## Data Clean

In [4]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [5]:
df = df.dropna(subset=['SOLD DATE'])

In [49]:
# Define list of desired months (excluding current month)
desired_months = ['April']

# Filter DataFrame to include only entries from desired months
df_filtered = df[df['SOLD DATE'].str.split('-', expand=True)[0].isin(desired_months)]

# Reset the index
df_filtered = df_filtered.reset_index(drop=True)

In [50]:
# Data checks
print(df_filtered['PRICE'].isna().value_counts())
print('-------')
print(df_filtered['$/SQUARE FEET'].isna().value_counts())
print('-------')
print(df_filtered['YEAR BUILT'].isna().value_counts())
print('-------')

PRICE
False    11656
Name: count, dtype: int64
-------
$/SQUARE FEET
False    11332
True       324
Name: count, dtype: int64
-------
YEAR BUILT
False    11416
True       240
Name: count, dtype: int64
-------


In [51]:
df_filtered.loc[df_filtered['PRICE'] == '0']

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,Unnamed: 0,URL.2,PRICE_AS_CURRENCY,YEAR BUILT DISPLAY,PRICE_SQUARE_FEET_AS_CURRENCY,COLOR,RANK,FULL_ADDRESS,Unnamed: 0.1,URL.1


In [52]:
df_filtered['PRICE'] = pd.to_numeric(df_filtered['PRICE'])
df_filtered['$/SQUARE FEET'] = pd.to_numeric(df_filtered['$/SQUARE FEET'])
df_filtered['YEAR BUILT'] = pd.to_numeric(df_filtered['YEAR BUILT'])
df_filtered['LATITUDE'] = pd.to_numeric(df_filtered['LATITUDE'])
df_filtered['LONGITUDE'] = pd.to_numeric(df_filtered['LONGITUDE'])

In [53]:
df_filtered.sort_values(by='PRICE',ascending=True).head(20)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,Unnamed: 0,URL.2,PRICE_AS_CURRENCY,YEAR BUILT DISPLAY,PRICE_SQUARE_FEET_AS_CURRENCY,COLOR,RANK,FULL_ADDRESS,Unnamed: 0.1,URL.1
10045,NaN,April-27-2023,NaN,547 W Stanton Ln,Crete,NaN,NaN,46600.0,3.0,NaN,...,NaN,NaN,"$46,600",1966.0,$45,blue,8740.0,547 W Stanton Ln Crete,NaN,NaN
10046,NaN,April-27-2023,NaN,547 W Stanton Ln,Crete,NaN,NaN,46600.0,3.0,NaN,...,2390.0,https://www.redfin.com/IL/Crete/547-W-Stanton-...,"$46,600",1966.0,$45,blue,8741.0,547 W Stanton Ln Crete,NaN,NaN
10047,NaN,April-27-2023,NaN,547 W Stanton Ln,Crete,NaN,NaN,46600.0,3.0,NaN,...,NaN,NaN,"$46,600",1966.0,$45,blue,8742.0,547 W Stanton Ln Crete,NaN,NaN
11132,PAST SALE,April-27-2023,Single Family Residential,547 W Stanton Ln,Crete,IL,60417.0,46600.0,3.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10747,PAST SALE,April-28-2023,Single Family Residential,137 Park Rd,Lakemoor,IL,60051.0,50000.0,2.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10043,NaN,April-28-2023,NaN,137 Park Rd,Lakemoor,NaN,NaN,50000.0,2.0,1.0,...,NaN,NaN,"$50,000",1940.0,$59,blue,8738.0,137 Park Rd Lakemoor,NaN,NaN
10044,NaN,April-28-2023,NaN,137 Park Rd,Lakemoor,NaN,NaN,50000.0,2.0,1.0,...,NaN,NaN,"$50,000",1940.0,$59,blue,8739.0,137 Park Rd Lakemoor,NaN,NaN
10042,NaN,April-28-2023,NaN,137 Park Rd,Lakemoor,NaN,NaN,50000.0,2.0,1.0,...,2005.0,https://www.redfin.com/IL/Lakemoor/137-Park-Rd...,"$50,000",1940.0,$59,blue,8737.0,137 Park Rd Lakemoor,NaN,NaN
10040,NaN,April-26-2023,NaN,3305 Sunset Ave #329,Waukegan,NaN,NaN,55001.0,1.0,1.0,...,NaN,NaN,"$55,001",1997.0,$122,blue,8735.0,3305 Sunset Ave #329 Waukegan,NaN,NaN
10039,NaN,April-26-2023,NaN,3305 Sunset Ave #329,Waukegan,NaN,NaN,55001.0,1.0,1.0,...,NaN,NaN,"$55,001",1997.0,$122,blue,8734.0,3305 Sunset Ave #329 Waukegan,NaN,NaN


In [54]:
print(df_filtered['URL'].iloc[682])

URL    https://www.redfin.com/IL/Naperville/106-N-Whi...
URL                                                  NaN
Name: 682, dtype: object


In [12]:
# # Correct the prices, if needed
# df_filtered.at[6,'PRICE']=(742000)
# df_filtered.at[708,'PRICE']=(749000)
# df_filtered.at[858,'PRICE']=(1100000)
# df_filtered.at[49,'PRICE']=(310000)
# df_filtered.at[152,'PRICE']=(650000)

In [55]:
# Find problem psf by searching for a '0' value
df_filtered.loc[df_filtered['$/SQUARE FEET'] == '0'][['SOLD DATE','ADDRESS','CITY','$/SQUARE FEET','PRICE','SQUARE FEET']]

,SOLD DATE,ADDRESS,CITY,$/SQUARE FEET,PRICE,SQUARE FEET


In [14]:
# # # Corrections, if needed
# df_filtered.at[708,'$/SQUARE FEET']=(749000/959)
# df_filtered.at[858,'$/SQUARE FEET']=(1100000/1251)
# df_filtered.at[6,'$/SQUARE FEET']=(742000/1152)
# df_filtered.at[49,'$/SQUARE FEET']=(310000/1215)
# df_filtered.at[152,'$/SQUARE FEET']=(650000/1487)
# df_filtered.at[682,'$/SQUARE FEET']=(85000/902)

In [56]:
# Find problem psf by searching for low values
df_filtered.sort_values(by='$/SQUARE FEET',ascending=True).head(20)[['PRICE','ADDRESS','CITY','$/SQUARE FEET']]

,PRICE,ADDRESS,CITY,$/SQUARE FEET
5388,350000.0,14700 W Kazmer Rd,Wadsworth,8.0
10365,350000.0,14700 W Kazmer Rd,Wadsworth,8.0
5351,350000.0,14700 W Kazmer Rd,Wadsworth,8.0
5396,350000.0,14700 W Kazmer Rd,Wadsworth,8.0
9190,180000.0,612 S Webster St,Aurora,18.0
9184,180000.0,612 S Webster St,Aurora,18.0
1113,180000.0,612 S Webster St,Aurora,18.0
9201,180000.0,612 S Webster St,Aurora,18.0
10023,65000.0,209 Oscar Ave,Joliet,34.0
10020,65000.0,209 Oscar Ave,Joliet,34.0


In [57]:
print(df_filtered.URL.iloc[1623])

URL                                                  NaN
URL    https://www.redfin.com/IL/Hinsdale/636-S-Stoug...
Name: 1623, dtype: object


In [58]:
# # Drop sales that aren't condos but labeled as such
# df_filtered = df_filtered.drop(1320)

In [60]:
df_filtered['PROPERTY TYPE'].value_counts()

PROPERTY TYPE
Single Family Residential    2106
Townhouse                     499
Condo/Co-op                   309
Name: count, dtype: int64

## Make Maps

In [18]:
### Create a price column formatted as currency ###
df_filtered['PRICE_AS_CURRENCY'] = df_filtered['PRICE'].apply(lambda x: "${:,.0f}".format(x))
### Set formatting for Beds, Baths ###
df_filtered['YEAR BUILT DISPLAY'] = df_filtered['YEAR BUILT'].apply(lambda x: '{:.0f}'.format(x))
df_filtered['PRICE_SQUARE_FEET_AS_CURRENCY'] = df_filtered['$/SQUARE FEET'].apply(lambda x: '${:,.0f}'.format(x))

In [19]:
df_filtered = df_filtered.sort_values(by=['PRICE'], ascending=False)
### Insert different colors for top 10 sales vs. the rest ###
df_filtered['COLOR'] = ''
### Create RANK column ###
df_filtered['RANK'] = 0
### Insert RANK values ###
df_filtered['RANK'] = range(1, len(df_filtered) + 1)
# use numpy to assign values to the 'COLOR' column
df_filtered['COLOR'] = np.where(df_filtered['RANK'] <= 10, 'orange', 'blue')

## HTML Popup Formatter

In [20]:
### Define list of columns to drop from DF ###
columns_drop = ['SALE TYPE','PROPERTY TYPE','STATE OR PROVINCE','ZIP OR POSTAL CODE','HOA/MONTH','STATUS','NEXT OPEN HOUSE START TIME','NEXT OPEN HOUSE END TIME','SOURCE','MLS#','FAVORITE','INTERESTED','SQUARE FEET','LOT SIZE']

In [21]:
### Drop the columns ###
df_filtered = df_filtered.drop(columns=columns_drop)

In [22]:
def popup_html(row):
    Price = row['PRICE_AS_CURRENCY']
    Address = row['ADDRESS']
    City = row['CITY']
    sold_date = row['SOLD DATE']
    beds = row['BEDS']
    baths = row['BATHS']
    psf = row['PRICE_SQUARE_FEET_AS_CURRENCY']
    year_built = row['YEAR BUILT DISPLAY']
    rank = row['RANK']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Price: </strong>{}'''.format(Price) + '''<br>
    <strong>Address: </strong>{}'''.format(Address) + '''<br>
    <strong>City: </strong>{}'''.format(City) + '''<br>
    <strong>Sold: </strong>{}'''.format(sold_date) + '''<br>
    <strong>Beds: </strong>{}'''.format(beds) + '''<br>
    <strong>Baths: </strong>{}'''.format(baths) + '''<br>
    <strong>Price per sf: </strong>{}'''.format(psf) + '''<br>
    <strong>Year Built: </strong>{}'''.format(year_built) + '''<br>
    <strong>Price Rank: </strong>{}'''.format(rank) + '''
    </html>
    '''
    return html

In [23]:
### Create map container ###
m = folium.Map(location=df_filtered[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=10,tiles=None)

### Create title ###
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f"April 2023 Collar County Single Family Sales")

m.get_root().html.add_child(folium.Element(title_html))

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top 10 Sales')

for index, row in df_filtered.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)

# Display map

In [24]:
m.save('index.html')

## Summary Info

In [25]:
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

Newest = '\033[1m' + 'Newest' + '\033[0m'
Oldest = '\033[1m' + 'Oldest' + '\033[0m'

In [26]:
df_filtered.columns

Index(['SOLD DATE', 'ADDRESS', 'CITY', 'PRICE', 'BEDS', 'BATHS', 'LOCATION',
       'YEAR BUILT', 'DAYS ON MARKET', '$/SQUARE FEET', 'URL', 'LATITUDE',
       'LONGITUDE', 'Unnamed: 0.2', 'URL', 'Unnamed: 0', 'URL.2',
       'PRICE_AS_CURRENCY', 'YEAR BUILT DISPLAY',
       'PRICE_SQUARE_FEET_AS_CURRENCY', 'COLOR', 'RANK', 'FULL_ADDRESS',
       'Unnamed: 0.1', 'URL.1'],
      dtype='object')

In [27]:
df_filtered['FULL_ADDRESS'] = df_filtered['ADDRESS'] + ' ' + df_filtered['CITY']

In [28]:
print(df_filtered.loc[df_filtered['PRICE'].idxmin()]['URL'])

URL                                                  NaN
URL    https://www.redfin.com/IL/Crete/547-W-Stanton-...
Name: 7132, dtype: object


In [29]:
print(f"{ME}{BR}{df_filtered.loc[df_filtered['PRICE'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['PRICE'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['PRICE'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['PRICE'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['PRICE'].idxmax()]['YEAR BUILT']:.0f}")
print(f"{LE}{BR}{df_filtered.loc[df_filtered['PRICE'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['PRICE'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['PRICE'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['PRICE'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['PRICE'].idxmin()]['YEAR BUILT']:.0f}")

print(f"{MAX_PSF}{BR}{df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['YEAR BUILT']:.0f}")
print(f"{MIN_PSF}{BR}{df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['YEAR BUILT']:.0f}")

print(f"{Newest}{BR}{df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['YEAR BUILT']:.0f}")
print(f"{Oldest}{BR}{df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['YEAR BUILT']:.0f}")

Most Expensive
Lake Forest, 172 W Laurel Ave Lake Forest | Price $3,600,000 | $551 psf | Year built: 2007
Least Expensive
Crete, 547 W Stanton Ln Crete | Price $46,600 | $45 psf | Year built: 1966
Highest Price Per Square Foot
Lake Forest, 172 W Laurel Ave Lake Forest | Price $3,600,000 | $551 psf | Year built: 2007
Lowest Price Per Square Foot
Old Mill Creek / Wadsworth, 14700 W Kazmer Rd Wadsworth | Price $350,000 | $8 psf | Year built: 1981
Newest
Westleigh Farm, 185 S Orchard Cir Lake Forest | Price $1,900,000 | $375 psf | Year built: 2023
Oldest
Glen Ellyn, 367 Duane St Glen Ellyn | Price $265,000 | $144 psf | Year built: 1856


## Time on Market Calculator

In [30]:
# print(df_Current.loc[df_Current['YEAR BUILT'].idxmin()])

In [31]:
from datetime import datetime, timedelta

date1 = datetime(2022, 2, 23) ## List (Earlier) date
date2 = datetime(2023, 4, 11) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

412


## Map URL Snagger

In [32]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

In [33]:
cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/CollarCounty_single_family_sales_month_ending_april_2023


## Get Summary Data

In [34]:
print('SALES INFO')
print(f'Number of sales: {len(df_filtered)}')
print('--------')
print(f'Total sale price: ${df_filtered["PRICE"].sum():,.0f}')
print('--------')
print(f'Median sale price: ${df_filtered["PRICE"].median():,.0f}')
print('--------')
print(f'Max sale price: ${df_filtered["PRICE"].max():,.0f}')
print('--------')
print(f'Min sale price: ${df_filtered["PRICE"].min():,.0f}')
print('------------------------------------------------')
print('PSF INFO')
print(f'Max price per square foot: ${df_filtered["$/SQUARE FEET"].max():,.0f}')
print('--------')
print(f'Min price per square foot: ${df_filtered["$/SQUARE FEET"].min():,.0f}')
print('--------')
print(f'Median price per square foot: ${df_filtered["$/SQUARE FEET"].median():,.0f}')
print('------------------------------------------------')
print('CONDO AGES')
print(f'Newest building: {df_filtered["YEAR BUILT"].max()}')
print('----------')
print(f'Oldest building: {df_filtered["YEAR BUILT"].min()}')
print('----------')
print(f'Average building age: {df_filtered["YEAR BUILT"].mean()}')

SALES INFO
Number of sales: 8742
--------
Total sale price: $3,435,252,345
--------
Median sale price: $335,000
--------
Max sale price: $3,600,000
--------
Min sale price: $46,600
------------------------------------------------
PSF INFO
Max price per square foot: $551
--------
Min price per square foot: $8
--------
Median price per square foot: $183
------------------------------------------------
CONDO AGES
Newest building: 2023.0
----------
Oldest building: 1856.0
----------
Average building age: 1982.2151366503153


In [37]:
df_filtered.columns

Index(['SOLD DATE', 'ADDRESS', 'CITY', 'PRICE', 'BEDS', 'BATHS', 'LOCATION',
       'YEAR BUILT', 'DAYS ON MARKET', '$/SQUARE FEET', 'URL', 'LATITUDE',
       'LONGITUDE', 'Unnamed: 0.2', 'URL', 'Unnamed: 0', 'URL.2',
       'PRICE_AS_CURRENCY', 'YEAR BUILT DISPLAY',
       'PRICE_SQUARE_FEET_AS_CURRENCY', 'COLOR', 'RANK', 'FULL_ADDRESS',
       'Unnamed: 0.1', 'URL.1'],
      dtype='object')

In [35]:
df_filtered.to_csv('April_CollarCounty.csv')